In [1]:
import os

In [2]:
pwd

'/Users/salonisahal/Desktop/Text-Summarizer-/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/salonisahal/Desktop/Text-Summarizer-'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directory

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH, 
            params_filepath=PARAMS_FILE_PATH):
            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)
            create_directory([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directory([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )

          return data_ingestion_config

In [8]:
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config
    

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers =request.urlretrieve(
                url= self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f" File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        '''
        zip_file_path:str
        extracts the zip file into the data directory
        function returns None
        '''
        unzip_path= self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion =DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-24 15:26:16,813: INFO: common:yaml file:config/config.yaml loaded Successfully]
[2025-04-24 15:26:16,817: INFO: common:yaml file:params.yaml loaded Successfully]
[2025-04-24 15:26:16,818: INFO: common:Directory artifacts created successfully.]
[2025-04-24 15:26:16,819: INFO: common:Directory artifacts/data_ingestion created successfully.]
[2025-04-24 15:26:19,165: INFO: 1807293343:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 167C:1E776:34D83A:835A91:680A0ABE
Accept-Ranges: bytes
Date: Thu, 24 Apr 2025 09:56:18 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830042-C